In [ ]:
import numpy as np
import cv2
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from skimage.feature import hog
import glob
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip

%matplotlib inline
from IPython.display import HTML


class DataStore():
    def __init__(self):
        
        print('Loading data...')
        t1 = time.time()
        
        self.car_fnames = glob.glob('images/vehicles/*/*.png')
        self.scn_fnames = glob.glob('images/non-vehicles/*/*.png')
        self.car_images = [mpimg.imread(fname) for fname in self.car_fnames]
        self.scn_images = [mpimg.imread(fname) for fname in self.scn_fnames]

        t2 = time.time()
        print('Loaded all data in {} seconds'.format(round(t2-t1), 2))
        

class VehicleDetection():


    def __init__(self, data_store):

        self.color_space = 'HLS'
        self.spatial_size = (32, 32)
        self.hist_bins = 32
        self.orient = 9
        self.pix_per_cell = 8
        self.cell_per_block = 2
        self.hog_channel = 'ALL'
        self.spatial_feat = False
        self.hist_feat = False
        self.hog_feat = True        
        
        self.window_config = [
            {
                'y_start_stop': [400, None],
                'xy_window': (80, 80),
                'xy_overlap': (0.8, 0.8)
            },
            {
                'y_start_stop': [400, None],
                'xy_window': (120, 120),
                'xy_overlap': (0.8, 0.8)
            },
            {
                'y_start_stop': [400, None],
                'xy_window': (160, 160),
                'xy_overlap': (0.8, 0.8)
            },
            {
                'y_start_stop': [400, None],
                'xy_window': (24, 240),
                'xy_overlap': (0.8, 0.8)
            }
        ]
        
        print('Extracting features...')
        t1 = time.time()        
        
        self.car_images = data_store.car_images
        self.scn_images = data_store.scn_images
        
        car_features = self.extract_features(self.car_images)
        scn_features = self.extract_features(self.scn_images)

        unscaled_X = np.vstack((car_features, scn_features)).astype(np.float64)
        self.scaler = StandardScaler().fit(unscaled_X)
        
        self.X = self.scaler.transform(unscaled_X)
        self.y = np.hstack((np.ones(len(car_features)), np.zeros(len(scn_features))))
        
        t2 = time.time()
        print('Extracted all features in {} seconds'.format(round(t2-t1), 2))
        
    
    # Define a function to return HOG features and visualization
    def get_hog_features(self, img, vis=False, feature_vec=True):
        # Call with two outputs if vis==True
        if vis == True:
            features, hog_image = hog(img, orientations=self.orient, 
                                      pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                                      cells_per_block=(self.cell_per_block, self.cell_per_block), 
                                      transform_sqrt=True, 
                                      visualise=vis, feature_vector=feature_vec)
            return features, hog_image
        # Otherwise call with one output
        else:      
            features = hog(img, orientations=self.orient, 
                           pixels_per_cell=(self.pix_per_cell, self.pix_per_cell),
                           cells_per_block=(self.cell_per_block, self.cell_per_block), 
                           transform_sqrt=True, 
                           visualise=vis, feature_vector=feature_vec)
            return features


    # Define a function to compute binned color features  
    def bin_spatial(self, img):
        # Use cv2.resize().ravel() to create the feature vector
        features = cv2.resize(img, self.spatial_size).ravel() 
        # Return the feature vector
        return features


    # Define a function to compute color histogram features 
    # NEED TO CHANGE bins_range if reading .png files with mpimg!
    def color_hist(self, img, bins_range=(0, 256)):
        # Compute the histogram of the color channels separately
        channel1_hist = np.histogram(img[:,:,0], bins=self.hist_bins, range=bins_range)
        channel2_hist = np.histogram(img[:,:,1], bins=self.hist_bins, range=bins_range)
        channel3_hist = np.histogram(img[:,:,2], bins=self.hist_bins, range=bins_range)
        # Concatenate the histograms into a single feature vector
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
        # Return the individual histograms, bin_centers and feature vector
        return hist_features


    # Define a function to extract features from a list of images
    # Have this function call bin_spatial() and color_hist()
    def extract_features(self, imgs):
        # Create a list to append feature vectors to
        features = []
        # Iterate through the list of images
        for image in imgs:
            file_features = []
            # Read in each one by one
            # apply color conversion if other than 'RGB'
            if self.color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif self.color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif self.color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif self.color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif self.color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
            else:
                feature_image = np.copy(image)      
    
            if self.spatial_feat == True:
                spatial_features = self.bin_spatial(feature_image)
                file_features.append(spatial_features)
            if self.hist_feat == True:
                # Apply color_hist()
                hist_features = self.color_hist(feature_image)
                file_features.append(hist_features)
            if self.hog_feat == True:
            # Call get_hog_features() with vis=False, feature_vec=True
                if self.hog_channel == 'ALL':
                    hog_features = []
                    for channel in range(feature_image.shape[2]):
                        hog_features.append(self.get_hog_features(feature_image[:,:,channel], vis=False, feature_vec=True))
                    hog_features = np.ravel(hog_features)        
                else:
                    hog_features = self.get_hog_features(feature_image[:,:,self.hog_channel], vis=False, feature_vec=True)
                # Append the new feature vector to the features list
                file_features.append(hog_features)
            features.append(np.concatenate(file_features))
        # Return list of feature vectors
        return features


    # Define a function to extract features from a single image window
    # This function is very similar to extract_features()
    # just for a single image rather than list of images
    def single_img_features(self, img):
        #1) Define an empty list to receive features
        img_features = []
        #2) Apply color conversion if other than 'RGB'
        if self.color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif self.color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif self.color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif self.color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif self.color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else:
            feature_image = np.copy(img)      
        #3) Compute spatial features if flag is set
        if self.spatial_feat == True:
            spatial_features = self.bin_spatial(feature_image)
            #4) Append features to list
            img_features.append(spatial_features)
        #5) Compute histogram features if flag is set
        if self.hist_feat == True:
            hist_features = self.color_hist(feature_image)
            #6) Append features to list
            img_features.append(hist_features)
        #7) Compute HOG features if flag is set
        if self.hog_feat == True:
            if self.hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.extend(self.get_hog_features(feature_image[:,:,channel], vis=False, feature_vec=True))      
            else:
                hog_features = self.get_hog_features(feature_image[:,:,self.hog_channel], vis=False, feature_vec=True)
            #8) Append features to list
            img_features.append(hog_features)
        #9) Return concatenated array of features
        return np.concatenate(img_features)


    # Define a function that takes an image,
    # start and stop positions in both x and y, 
    # window size (x and y dimensions),  
    # and overlap fraction (for both x and y)
    def slide_window(self, img, x_start_stop=[None, None], y_start_stop=[None, None], 
                     xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
        # If x and/or y start/stop positions not defined, set to image size
        if x_start_stop[0] == None:
            x_start_stop[0] = 0
        if x_start_stop[1] == None:
            x_start_stop[1] = img.shape[1]
        if y_start_stop[0] == None:
            y_start_stop[0] = 0
        if y_start_stop[1] == None:
            y_start_stop[1] = img.shape[0]
        # Compute the span of the region to be searched    
        xspan = x_start_stop[1] - x_start_stop[0]
        yspan = y_start_stop[1] - y_start_stop[0]
        # Compute the number of pixels per step in x/y
        nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
        ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
        # Compute the number of windows in x/y
        nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
        ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
        nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
        ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
        # Initialize a list to append window positions to
        window_list = []
        # Loop through finding x and y window positions
        # Note: you could vectorize this step, but in practice
        # you'll be considering windows one by one with your
        # classifier, so looping makes sense
        for ys in range(ny_windows):
            for xs in range(nx_windows):
                # Calculate window position
                startx = xs*nx_pix_per_step + x_start_stop[0]
                endx = startx + xy_window[0]
                starty = ys*ny_pix_per_step + y_start_stop[0]
                endy = starty + xy_window[1]
                
                # Append window position to list
                window_list.append(((startx, starty), (endx, endy)))
        # Return the list of windows
        return window_list

    
    def multiplex_windows(self, img):
        windows = []
        for cfg in self.window_config:
            windows += self.slide_window(img,
                                         y_start_stop=cfg['y_start_stop'],
                                         xy_window=cfg['xy_window'],
                                         xy_overlap=cfg['xy_overlap'])
        return windows
        
    

    # Define  function you will pass an image 
    # and the list of windows to be searched (output of slide_windows())
    def search_windows(self, img, windows):
        #1) Create an empty list to receive positive detection windows
        on_windows = []
        #2) Iterate over all windows in the list
        for window in windows:
            #3) Extract the test window from original image
            sub_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
            #4) Extract features for that window using single_img_features()
            features = self.single_img_features(sub_img)
            #5) Scale extracted features to be fed to classifier
            features = self.scaler.transform(np.array(features).reshape(1, -1))
            #6) Predict using your classifier
            prediction = self.svc.predict(features)
            #7) If positive (prediction == 1) then save the window
            if prediction == 1:
                on_windows.append(window)
        #8) Return windows for positive detections
        return on_windows


    def draw_boxes(self, img, bboxes, color=(0, 0, 255), thick=6):
        # Make a copy of the image
        imcopy = np.copy(img)
        # Iterate through the bounding boxes
        for bbox in bboxes:
            # Draw a rectangle given bbox coordinates
            cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
        # Return the image copy with boxes drawn
        return imcopy

            
    def shuffle_and_train(self):
        print('Training SVM...')
        t1 = time.time()
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=np.random.randint(0, 100))
        self.svc = LinearSVC()
        self.svc.fit(self.X_train, self.y_train)
        score = vd.svc.score(vd.X_test, vd.y_test)
        t2 = time.time()
        print('Trained SVM in {} seconds with {} accuracy'.format(round(t2-t1, 2), score))



In [ ]:
ds = DataStore()

In [ ]:
vd = VehicleDetection(ds)
vd.shuffle_and_train()

In [ ]:
img = mpimg.imread('test_images/test1.jpg')

windows = vd.multiplex_windows(img)
hot_windows = vd.search_windows(img, windows)

boxed_img = vd.draw_boxes(np.copy(img), hot_windows)
plt.imshow(boxed_img)
